<a href="https://colab.research.google.com/github/daraseliya/course_date_engineer/blob/develop/Task_4_1_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
# Импорт библиотек


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings("ignore") 
from sklearn import ensemble
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV , train_test_split , cross_val_score
import sklearn
from sklearn import svm 
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
    

In [52]:
from sklearn.neighbors import KNeighborsClassifier

In [42]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [45]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [55]:
from sklearn.neighbors import RadiusNeighborsClassifier

In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

#Работа с данными:


## 1. Импорт с kaggle

In [24]:
df = pd.read_csv("/content/train.csv")

#dfTest = pd.read_csv("/content/test.csv",index_col="PassengerId")
#dfTrain = pd.read_csv("/content/train.csv",index_col="PassengerId")

In [25]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## 2. Удаление колонок, не подходящих для обучения

In [26]:
df = df.drop(['PassengerId'], axis=1) #тк просто индексы
df = df.drop(['Name'], axis=1) # имена уникальны, просто string данные
df = df.drop(['Ticket'], axis=1) # строка
df = df.drop(['Cabin'], axis=1) # тоже строка

#для обучения подойдут survived (target), Pclass, пол, возраст и Fare

## 3. Убираем пропуски - пустые ячейки

In [27]:
df['Age'] =  df['Age'].fillna(df['Age'].median())
df['Embarked'] =  df['Embarked'].fillna(df['Embarked'].mode())
df['Fare'] =  df['Fare'].fillna(df['Fare'].mean())

In [28]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,28.0,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [ ]:
round(df.groupby("Pclass").Fare.mean(),2)

Pclass
1    84.15
2    20.66
3    13.68
Name: Fare, dtype: float64

# Работа с некатегориальными и категориальными признаками 
### (используется sklearn.preprocessing.StandardScaler)

## 1. Некатегориальные признаки

In [30]:
df_cat = df[['Survived','Pclass', 'Sex', 'Embarked']]
df_num =df[['Age', 'SibSp', 'Parch', 'Fare']]
df_col = ['Age', 'SibSp', 'Parch', 'Fare']

scaler = StandardScaler()
df_num = scaler.fit_transform(df_num)
df_num = pd.DataFrame(df_num)
df_num.columns = df_col

df_final = df_num.join(df_cat)

## 2. Категориальные признаки

In [ ]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(df_final[['Sex']])

def get_ohe(df_f):
    temp_df = pd.DataFrame(data=ohe.transform(df_f[['Sex']]), columns=ohe.get_feature_names_out())
    df_f.drop(columns=['Sex'], axis=1, inplace=True)
    df_f = pd.concat([df_f.reset_index(drop=True), temp_df], axis=1)
    return df_f

df_final = get_ohe(df_final)

In [33]:
ohe.fit(df_final[['Embarked']])
def get_ohe(df_f):
    temp_df = pd.DataFrame(data=ohe.transform(df_f[['Embarked']]), columns=ohe.get_feature_names_out())
    df_f.drop(columns=['Embarked'], axis=1, inplace=True)
    df_f = pd.concat([df_f.reset_index(drop=True), temp_df], axis=1)
    return df_f
df_final = get_ohe(df_final)

df_final = df_final.drop(['Sex_female', 'Embarked_C'], axis=1)

# Разделение на тестирование и обучение

In [34]:
X = df_final.drop('Survived', axis=1)
y = df_final['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11)

# 1. KNN - модель ближайших соседей

In [35]:
#knn = KNeighborsRegressor() - нужен классиф., не регр.
# knn.fit(X_train, y_train)

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=3)

In [36]:
prediction_1 = model.predict(X_test)

In [37]:
from sklearn import metrics
KNN_accuracy = accuracy_score(y_test, prediction_1)

print('KNN accuracy: {}'.format(KNN_accuracy))

KNN accuracy: 0.8044692737430168


#### (то есть примерно 80% на тесте были классифицированы правильно)





### Вычисление MSE, MAE, R2



In [41]:
for X, y, label in zip([X_train, X_test], [y_train, y_test], ['train', 'test']):
  pred = model.predict(X)
  print(f'MSE {label}={mean_squared_error(y, pred):.2f}')
  print(f'MAE {label}={mean_absolute_error(y, pred):.2f}')
  print(f'R2 {label}={r2_score(y, pred):.2f}')    

MSE train=0.13
MAE train=0.13
R2 train=0.47
MSE test=0.20
MAE test=0.20
R2 test=0.17


### (Метрики получились не очень, но у нас используется классификатор, а не регрессор, поэтому MSE, MAE, R2 вычислять не нужно)

In [44]:
precision_recall_fscore_support(y_test, prediction_1)

(array([0.82051282, 0.77419355]),
 array([0.87272727, 0.69565217]),
 array([0.84581498, 0.73282443]),
 array([110,  69]))

# Обучение, настройка (подбор) параметров

In [ ]:
#params = dict(
#    n_neighbors=range(1, 40),
#    weights=['uniform', 'distance'],
#    p=[1, 2],
#
#)
# ---------вычисления для регруссора, тут не используется

In [46]:
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

In [ ]:
#model2 = KNeighborsRegressor()
#cv = GridSearchCV(knn, params, n_jobs=-1, cv=5)
#cv.fit(X_train, y_train)
# ---------вычисления для регруссора, тут не используется

GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': range(1, 40), 'p': [1, 2],
                         'weights': ['uniform', 'distance']})

In [49]:
model2 = KNeighborsClassifier()

Randomized_Search = RandomizedSearchCV(estimator=model2, param_distributions = hyperparameters,
                               cv = 5, n_iter = 10, n_jobs=-1)
Randomized_Search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
                   param_distributions={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...],
                                        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19, 20,
                                                        21, 22, 23, 24, 25, 26,
                                                        27, 28, 29],
                                        'p': [1, 2]})

### Посмотрим лучшие параметры:

In [ ]:
#best_params = cv.best_params_
#best_params
# ---------вычисления для регруссора, тут не используется

{'n_neighbors': 8, 'p': 1, 'weights': 'uniform'}

In [51]:
print("\n Лучший оценщик:", Randomized_Search.best_estimator_)
print("\n Лучшая оценка:", Randomized_Search.best_score_)
print("\n Лучшие параметры:", Randomized_Search.best_params_)


 Лучший оценщик: KNeighborsClassifier(leaf_size=9, n_neighbors=24, p=1)

 Лучшая оценка: 0.8173643258150299

 Лучшие параметры: {'p': 1, 'n_neighbors': 24, 'leaf_size': 9}


### Теперь подставим параметры, которые определили как более подходящие

In [ ]:
#knn = KNeighborsRegressor(**best_params)
#knn.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=8, p=1)

In [53]:
model = KNeighborsClassifier(leaf_size=9, n_neighbors=24, p=1)
model.fit(X_train, y_train)
prediction_1 = model.predict(X_test)

In [54]:
KNN_accuracy = accuracy_score(y_test, prediction_1)

print('KNN accuracy: {}'.format(KNN_accuracy))

KNN accuracy: 0.8324022346368715


#### (то есть примерно 83% были классифицированы правильно, до этого (выше, до побора параметров) точность составляла 80% )

# 2. RadiusNeighborsClassifier

In [59]:
from sklearn.neighbors import RadiusNeighborsClassifier

In [56]:
model3 = RadiusNeighborsClassifier(radius=1.0)
model3.fit(X_train, y_train)

RadiusNeighborsClassifier()

In [ ]:
# не подойдет

# 3. RandomForestClassifier

In [63]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [64]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [67]:
prediction_3 = clf.predict(X_test)

In [69]:
RFC_accuracy = accuracy_score(y_test, prediction_3)

print('RFC accuracy: {}'.format(RFC_accuracy))

RFC accuracy: 0.8212290502793296


#### (то есть примерно 82% на тесте были классифицированы правильно, в KNN было только 80%)

In [70]:
precision_recall_fscore_support(y_test, prediction_3)

(array([0.77857143, 0.97435897]),
 array([0.99090909, 0.55072464]),
 array([0.872    , 0.7037037]),
 array([110,  69]))

# Обучение, настройка (подбор) параметров

In [85]:
#leaf_size = list(range(1,50))
n_estimators = list(range(10,500))
n_features= list(range(1,10))
random_state=[0]
max_depth=[1,2,3,4,5]
hyperparameters2 = dict(n_estimators=n_estimators, max_depth=max_depth)

In [86]:
model4 =  RandomForestClassifier()

Randomized_Search2 = RandomizedSearchCV(estimator=model4, param_distributions = hyperparameters2, random_state=0)
Randomized_Search2.fit(X_train, y_train)

RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': [1, 2, 3, 4, 5],
                                        'n_estimators': [10, 11, 12, 13, 14, 15,
                                                         16, 17, 18, 19, 20, 21,
                                                         22, 23, 24, 25, 26, 27,
                                                         28, 29, 30, 31, 32, 33,
                                                         34, 35, 36, 37, 38, 39, ...]},
                   random_state=0)

### Посмотрим лучшие параметры:

In [87]:
print("\n Лучший оценщик:", Randomized_Search2.best_estimator_)
print("\n Лучшая оценка:", Randomized_Search2.best_score_)
print("\n Лучшие параметры:", Randomized_Search2.best_params_)


 Лучший оценщик: RandomForestClassifier(max_depth=4, n_estimators=271)

 Лучшая оценка: 0.8286417807544566

 Лучшие параметры: {'n_estimators': 271, 'max_depth': 4}


### Теперь подставим параметры, которые определили как более подходящие

In [88]:
model5 = RandomForestClassifier(n_estimators=271, max_depth=4, random_state=0)
model5.fit(X_train, y_train)
prediction_4 = model5.predict(X_test)

In [89]:
RFC_accuracy = accuracy_score(y_test, prediction_4)

print('RFC accuracy: {}'.format(RFC_accuracy))

RFC accuracy: 0.8435754189944135


###  (то есть примерно 84% были классифицированы правильно (было 82%), для KHH точность составила 83% )